In [1]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
import tensorflow as tf


os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "2"



Using TensorFlow backend.


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
import numpy as np
import pandas as pd
import glob
import csv
import sys
import os
import math
import tensorflow as tf
import keras
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Embedding
from keras.layers import LSTM
from keras.datasets import imdb
from keras.callbacks import EarlyStopping
from sklearn import metrics
from keras.layers.core import Dense, Activation
from keras.callbacks import ModelCheckpoint
from numpy import insert
from sklearn import preprocessing
from collections import Counter
from numpy import array
from numpy import hstack
from keras.preprocessing.sequence import TimeseriesGenerator




path  = r"/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/Buildserver/" 

names = ['Operation','TimeStamp','Process_Name','ThreadID','IrpPtr','ByteOffset','IOSize','ThreadID','ElapsedTime','DiskNum','IrpFlags','DiskSvcTime','I/O Pri','VolSnap','FileObject','FileName','IO_Pri']



all_files = glob.glob(os.path.join(path, "data-output-buildserver-2_total.csv"))
print(all_files)




['/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/Buildserver/data-output-buildserver-2_total.csv']


In [4]:
df = pd.read_csv(all_files[0],engine='python',skiprows =1,header=None,na_values=['-1'], index_col=False) 
names = ['Operation','TimeStamp','Process_Name','ThreadID','IrpPtr','ByteOffset','IOSize','ThreadID','ElapsedTime','DiskNum','IrpFlags','DiskSvcTime','I/O Pri','VolSnap','FileObject','FileName','IO_Pri']
df.columns = names
print (len(df))

1600438


In [5]:
#Sorting df by TimeStamp

df = df.sort_values(by=['TimeStamp'])
df.reset_index(inplace=True, drop=True)
print(df.dtypes)
print(len(df))

Operation        object
TimeStamp        object
Process_Name     object
ThreadID         object
IrpPtr           object
ByteOffset       object
IOSize           object
ThreadID         object
ElapsedTime      object
DiskNum          object
IrpFlags         object
DiskSvcTime      object
I/O Pri         float64
VolSnap          object
FileObject       object
FileName         object
IO_Pri           object
dtype: object
1600438


In [6]:
addresses = df['ByteOffset'].tolist()
size = df['IOSize'].tolist()
addresses_dec = []
size_dec = []
count = 0

while (count < len(addresses)):
    if "Offset" in addresses[count]:
        count = count +1
        continue
    dec = addresses[count]
    dec_size = addresses[count]
    tmp = int(dec, 16)
    tmp_size = int(dec_size,16)
    addresses_dec.append(tmp)
    size_dec.append(tmp_size)
    count = count +1
    
print(len(addresses_dec))
print(len(size_dec))




1600430
1600430


In [7]:
from collections import Counter
print(len(Counter(df['IOSize'])))
print(len(Counter(df['ByteOffset'])))

630
1329410


In [9]:
df_a = pd.DataFrame(columns = ['ByteOffset', 'IOSize']) 

df_a['ByteOffset'] = addresses_dec
df_a['Size'] = size_dec

df = df_a

In [10]:
df.head(5)

,ByteOffset,IOSize,Size
0,251824893952,NaN,251824893952
1,319350947840,NaN,319350947840
2,81801076736,NaN,81801076736
3,186105143296,NaN,186105143296
4,240879329280,NaN,240879329280


In [11]:
df['ByteOffset_Delta'] = df['ByteOffset'] - df['ByteOffset'].shift(-1)
df = df.drop(df.index[-1])

In [12]:
df['ByteOffset_Delta'] = df['ByteOffset_Delta'].fillna(0)

a = df['ByteOffset_Delta'].unique().tolist()
operation_id_map = {}
for i,id in enumerate(a): operation_id_map[id] = i 
df['ByteOffset_Delta_class'] = df['ByteOffset_Delta'].map(lambda x: operation_id_map[x])


In [13]:
# a = df['IOSize'].unique().tolist()
# size_id_map = {}
# for i,id in enumerate(a): size_id_map[id] = i 
# df['Size_class'] = df['IOSize'].map(lambda x: size_id_map[x])

In [14]:
from collections import Counter
x = Counter(df['ByteOffset_Delta_class'])
vals = {}
vals =  x.most_common(1000)
bo_list = []

for x in vals:
    bo_list.append(x[0])
        
count = 0
label_list = []

while (count < len(df)):
    x = df['ByteOffset_Delta_class'].iloc[count]
    if x in bo_list:
        label_list.append(x)
    else:
        label_list.append(999999)
    count= count + 1
    
ByteOffset_Delta_class_backup  = df['ByteOffset_Delta_class'] 
df['ByteOffset_Delta_class']  = label_list
print(len(Counter(df['ByteOffset_Delta_class'])))
    
a = df['ByteOffset_Delta_class'].unique().tolist()
bo_map = {}
for i,id in enumerate(a): bo_map[id] = i 
df['ByteOffset_Delta_Class_1001'] = df['ByteOffset_Delta_class'].map(lambda x: bo_map[x])
    
label_list = df['ByteOffset_Delta_Class_1001'] 
    
df['ByteOffset_Delta_Class_1001']  = label_list
    

1001


In [15]:
len(Counter(df['ByteOffset_Delta_Class_1001']))

1001

In [16]:
df.head(5)

,ByteOffset,IOSize,Size,ByteOffset_Delta,ByteOffset_Delta_class,ByteOffset_Delta_Class_1001
0,251824893952,NaN,251824893952,-6.752605e+10,999999,0
1,319350947840,NaN,319350947840,2.375499e+11,999999,0
2,81801076736,NaN,81801076736,-1.043041e+11,999999,0
3,186105143296,NaN,186105143296,-5.477419e+10,999999,0
4,240879329280,NaN,240879329280,1.861192e+11,999999,0


In [17]:
from collections import Counter

df['IOSize_log'] = np.log2(df['Size'])

df['IOSize_log_roundoff']= round(df['IOSize_log'])
print(len(Counter(df['Size'])))
print(len(Counter(df['IOSize_log'])))
print(len(Counter(df['IOSize_log_roundoff'])))


1329409
1329409
18


In [18]:
a = df['IOSize_log_roundoff'].unique().tolist()
size_id_map = {}
for i,id in enumerate(a): size_id_map[id] = i 
df['Size_class'] = df['IOSize_log_roundoff'].map(lambda x: size_id_map[x])

In [19]:
len(Counter(df['ByteOffset_Delta_Class_1001']))
len(Counter(df['Size_class']))

18

In [20]:
# Split to train, validate and test

# Finding the value 75th percentile of TimeStamp

import math


training_pt_1 = math.floor((len(df)*0.75)) 

lba_train =df[:training_pt_1]['ByteOffset_Delta_Class_1001'].tolist()
lba_test = df[training_pt_1+1:]['ByteOffset_Delta_Class_1001'].tolist()
size_train = df[:training_pt_1]['Size_class'].tolist()
size_test = df[training_pt_1+1:]['Size_class'].tolist()


In [21]:
lba_train= np.array(lba_train).reshape(-1,1)
lba_test= np.array(lba_test).reshape(-1,1)
size_train= np.array(size_train).reshape(-1,1)
size_test= np.array(size_test).reshape(-1,1)

In [22]:
def create_dataset2(dataset, window_size):
    dataX, dataY = [], []
    for i in range(len(dataset) - 2 * window_size):
        a = dataset[i:(i + window_size), 0]
        #print(a)
        dataX.append(a)
        b = dataset[(i + window_size):(i + 2* window_size), 0]
        #print(b)
        dataY.append(b)
    return np.array(dataX), np.array(dataY)

lstm_num_timesteps = 32
    
X_train_lba, y_train_lba = create_dataset2(lba_train, lstm_num_timesteps)
X_test_lba, y_test_lba = create_dataset2(lba_test, lstm_num_timesteps)
X_train_size, y_train_size = create_dataset2(size_train, lstm_num_timesteps)
X_test_size, y_test_size = create_dataset2(size_test, lstm_num_timesteps)

In [23]:
X_train_lba.shape, y_train_lba.shape, X_test_lba.shape,  y_test_lba.shape

((1200257, 32), (1200257, 32), (400043, 32), (400043, 32))

In [24]:
X_train_size.shape, y_train_size.shape, X_test_size.shape,  y_test_size.shape

((1200257, 32), (1200257, 32), (400043, 32), (400043, 32))

In [25]:
import pickle as pkl


file__1 = open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver_2_X_train_lba.csv', 'wb')
pkl.dump(X_train_lba, file__1)
file__1.close()

file__1 = open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver_2_X_train_size.csv', 'wb')
pkl.dump(X_train_size, file__1)
file__1.close()


file__1 = open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver_2_X_test_lba.csv', 'wb')
pkl.dump(X_test_lba, file__1)
file__1.close()

file__1 = open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver_2_X_test_size.csv', 'wb')
pkl.dump(X_test_size, file__1)
file__1.close()


file__1 = open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver_2_y_train_lba.csv', 'wb')
pkl.dump(y_train_lba, file__1)
file__1.close()

file__1 = open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver_2_y_train_size.csv', 'wb')
pkl.dump(y_train_size, file__1)
file__1.close()

file__1 = open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver_2_y_test_lba.csv', 'wb')
pkl.dump(y_test_lba, file__1)
file__1.close()


file__1 = open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver_2_y_test_size.csv', 'wb')
pkl.dump(y_test_size, file__1)
file__1.close()



In [ ]:
# import pickle

# with open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver-2_X_train_lba.csv','rb') as f: X_train_lba = pickle.load(f)
# with open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver-2_X_train_size.csv','rb') as f: X_train_size = pickle.load(f)
# with open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver-2_X_train_lba.csv','rb') as f: X_test_lba = pickle.load(f)
# with open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver-2_X_train_lba.csv','rb') as f: X_test_size = pickle.load(f)

                
# with open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver-2_y_train_lba.csv','rb') as f: y_train_lba = pickle.load(f)
# with open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver-2_y_train_size.csv','rb') as f: y_train_size = pickle.load(f)
# with open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver-2_y_test_lba.csv','rb') as f: y_test_lba = pickle.load(f)
# with open('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/numpy_array_save/buildserver-2_y_test_size.csv','rb') as f: y_test_size = pickle.load(f)

In [26]:
X_train_lba.shape, y_train_lba.shape, X_test_lba.shape,  y_test_lba.shape

((1200257, 32), (1200257, 32), (400043, 32), (400043, 32))

In [27]:
X_train_size.shape, y_train_size.shape, X_test_size.shape,  y_test_size.shape

((1200257, 32), (1200257, 32), (400043, 32), (400043, 32))

In [28]:
lstm_num_features = 1
lstm_predict_sequences = True
lstm_num_predictions = 32


# X_train = np.reshape(X_train, (X_train.shape[0], lstm_num_timesteps, lstm_num_features))
# X_test = np.reshape(X_test, (X_test.shape[0], lstm_num_timesteps, lstm_num_features))
    

y_train_lba = np.reshape(y_train_lba, (y_train_lba.shape[0], lstm_num_predictions, lstm_num_features))
y_test_lba = np.reshape(y_test_lba, (y_test_lba.shape[0], lstm_num_predictions, lstm_num_features))
y_train_size = np.reshape(y_train_size, (y_train_size.shape[0], lstm_num_predictions, lstm_num_features))
y_test_size = np.reshape(y_test_size, (y_test_size.shape[0], lstm_num_predictions, lstm_num_features))                        


In [29]:
# Two classification outputs
import keras

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
from keras.datasets import imdb


from tensorflow.python.client import device_lib
import numpy as np
import csv
import pandas as pd
import sys
import os
import glob
import tensorflow as tf






from keras.layers import Dense, Input
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Concatenate, Dense, LSTM, Input, concatenate , Dot
from keras.callbacks import EarlyStopping
from keras.layers import Dense, Activation, Embedding, Dropout, TimeDistributed, Reshape




# no_docs = len(y_train_lba)
maxlen= 32



# # define two sets of inputs
# inputA = Input(shape=(32,))
# inputB = Input(shape=(32,))
# # inputA = Sequential()
# # inputB = Sequential()
vocabulary_1 = 1001
vocabulary_2 = len(Counter(df['Size_class']))

hidden_size = 500

# input=Input(shape=(no_docs,maxlen),dtype='float64')
inputA=Input(shape=(maxlen,),dtype='float64')  
inputB=Input(shape=(maxlen,),dtype='float64') 


# the first branch operates on the first input
x = Embedding(input_dim=vocabulary_1,output_dim=hidden_size,input_length=maxlen)(inputA)
x = Model(inputs=inputA, outputs=x)

# # the second branch opreates on the second input
y = Embedding(input_dim=vocabulary_2,output_dim=hidden_size,input_length=maxlen)(inputB)
y = Model(inputs=inputB, outputs=y)
# combine the output of the two branches
combined = keras.layers.concatenate([x.output, y.output])

lstm1 = LSTM(hidden_size,return_sequences=True)(combined)
lstm2 = LSTM(hidden_size, return_sequences=True)(lstm1)

# create classification output
offset = keras.layers.wrappers.TimeDistributed(Dense(units=vocabulary_1, activation='softmax'), name='offset')(lstm2)
iosize = keras.layers.wrappers.TimeDistributed(Dense(units=vocabulary_2, activation='softmax'), name='iosize')(lstm2)




model =Model([inputA,inputB],[offset,iosize]) # combining all into a Keras model

model.compile(optimizer='rmsprop',
              loss={'offset': 'sparse_categorical_crossentropy', 'iosize': 'sparse_categorical_crossentropy'},
              loss_weights={'offset': 2., 'iosize': 1.5},
              metrics={ 'offset': 'categorical_accuracy', 'iosize': 'categorical_accuracy'})
model.summary()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 32)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 32, 500)      500500      input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 32, 500)      9000        input_2[0][0]                    
____________________________________________________________________________________________

In [30]:
import time
num_epochs = 1000
batch_size = 32


monitor = EarlyStopping(monitor='loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
checkpointer = ModelCheckpoint(filepath="best_weights-buildserver-2.hdf5", verbose=1, save_best_only=True) # save best model
valid = [X_test_lba,X_test_size],[y_test_lba,y_test_size]


valid = [X_test_lba,X_test_size],[y_test_lba,y_test_size]

model.fit([X_train_lba,X_train_size],[y_train_lba,y_train_size],
          verbose=1,epochs=1000,callbacks=[monitor,checkpointer],batch_size = 64,validation_data = valid)


print("--- %s seconds ---" % (time.time() - start_time))

print('Done ...!')

#model.fit(X_train, y_train, nb_epoch = num_epochs, batch_size = batch_size)


/soe/hlitz/miniconda3/envs/py3_tf_gpu/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1200257 samples, validate on 400043 samples
Epoch 1/1000
1200257/1200257 [==============================] - 1991s 2ms/step - loss: 3.0795 - offset_loss: 0.5057 - iosize_loss: 1.3788 - offset_categorical_accuracy: 0.9983 - iosize_categorical_accuracy: 0.6620 - val_loss: 3.2603 - val_offset_loss: 0.5628 - val_iosize_loss: 1.4231 - val_offset_categorical_accuracy: 0.9966 - val_iosize_categorical_accuracy: 0.6638

Epoch 00001: val_loss improved from inf to 3.26029, saving model to best_weights-buildserver-2.hdf5
Epoch 2/1000
1200257/1200257 [==============================] - 1998s 2ms/step - loss: 2.8313 - offset_loss: 0.4302 - iosize_loss: 1.3139 - offset_categorical_accuracy: 0.9853 - iosize_categorical_accuracy: 0.6405 - val_loss: 3.5723 - val_offset_loss: 0.6478 - val_iosize_loss: 1.5177 - val_offset_categorical_accuracy: 0.9877 - val_iosize_categorical_accuracy: 0.5295

Epoch 00002: val_loss did not improve from 3.26029
Epoch 3/1000
1200257/1200257 [==========================

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1200257/1200257 [==============================] - 1991s 2ms/step - loss: 1.8865 - offset_loss: 0.2634 - iosize_loss: 0.9065 - offset_categorical_accuracy: 0.9548 - iosize_categorical_accuracy: 0.5348 - val_loss: 5.0604 - val_offset_loss: 0.8560 - val_iosize_loss: 2.2322 - val_offset_categorical_accuracy: 0.9645 - val_iosize_categorical_accuracy: 0.4453

Epoch 00014: val_loss did not improve from 3.26029
Epoch 15/1000
  67264/1200257 [>.............................] - ETA: 26:22 - loss: 1.8595 - offset_loss: 0.2593 - iosize_loss: 0.8940 - offset_categorical_accuracy: 0.9547 - iosize_categorical_accuracy: 0.5329

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



...

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



 925120/1200257 [======================>.......] - ETA: 7:13 - loss: 1.8038 - offset_loss: 0.2553 - iosize_loss: 0.8622 - offset_categorical_accuracy: 0.9534 - iosize_categorical_accuracy: 0.5223

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



1200257/1200257 [==============================] - 2304s 2ms/step - loss: 1.8049 - offset_loss: 0.2556 - iosize_loss: 0.8625 - offset_categorical_accuracy: 0.9534 - iosize_categorical_accuracy: 0.5223 - val_loss: 5.2545 - val_offset_loss: 0.8667 - val_iosize_loss: 2.3473 - val_offset_categorical_accuracy: 0.9695 - val_iosize_categorical_accuracy: 0.4673

Epoch 00020: val_loss did not improve from 3.26029
Epoch 21/1000
1200257/1200257 [==============================] - 2342s 2ms/step - loss: 1.7960 - offset_loss: 0.2548 - iosize_loss: 0.8576 - offset_categorical_accuracy: 0.9533 - iosize_categorical_accuracy: 0.5210 - val_loss: 5.2720 - val_offset_loss: 0.8657 - val_iosize_loss: 2.3603 - val_offset_categorical_accuracy: 0.9707 - val_iosize_categorical_accuracy: 0.4997

Epoch 00021: val_loss did not improve from 3.26029
Epoch 22/1000
 380160/1200257 [========>.....................] - ETA: 22:01 - loss: 1.7807 - offset_loss: 0.2522 - iosize_loss: 0.8508 - offset_categorical_accuracy: 0.95

KeyboardInterrupt: 

In [31]:
model.save('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/Model_checkpoints/buildserver-2_model.h5')

In [32]:
new_model = keras.models.load_model('/soe/hlitz/maxwell/notebooks/Prefetching_SSD/Data/output/Model_checkpoints/buildserver-2_model.h5')

/soe/hlitz/miniconda3/envs/py3_tf_gpu/lib/python3.6/site-packages/tensorflow_core/python/framework/indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [34]:
import time
start_time = time.time()

pred1,pred2 = new_model.predict([X_test_lba,X_test_size],verbose =1 )
end_time = time.time()
elapsed_time = end_time - start_time
print(elapsed_time)



400043/400043 [==============================] - 579s 1ms/step
579.3955948352814


In [35]:
import time
start_time = time.time()

pred_1 = pred1[:,0,:]
pred_2 = pred2[:,0,:]
pred_1 = np.argmax(pred_1, axis=1)
pred_2 = np.argmax(pred_2, axis=1)
elapsed_time = time.time() - start_time
print(elapsed_time)


5.65558934211731


In [46]:
# Split to train, validate and test

# Finding the value 75th percentile of TimeStamp

import math


training_pt_1 = math.floor((len(df)*0.75)) 

lba_train =df[:training_pt_1]['ByteOffset_Delta_Class_1001'].tolist()
lba_test = df[training_pt_1+1:]['ByteOffset_Delta_Class_1001'].tolist()
size_train = df[:training_pt_1]['Size_class'].tolist()
size_test = df[training_pt_1+1:]['Size_class'].tolist()


In [47]:
len(lba_test)

400107

In [48]:
print(len(pred1))
lba_test = lba_test[(-len(pred1)):]
print(len(lba_test))

400043
400043


In [49]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error

lba_test_final = lba_test[(-len(pred1)):]
lba_size_final = size_test[(-len(pred1)):]


lba_accuracy = accuracy_score(lba_test_final, pred_1)


size_accuracy = accuracy_score(lba_size_final, pred_2)

In [50]:
print(lba_accuracy)
print(size_accuracy)

0.9217834082836095
0.3561542134220571
